In [ ]:
pip install --upgrade pip

In [ ]:
pip install Pillow

In [ ]:
pip install ipywidgets

In [ ]:
pip install datasets

In [ ]:
pip install datasets[vision]

In [ ]:
# Accessing ImageNet from https://huggingface.co/datasets/ILSVRC/imagenet-1k

In [27]:
# pip uninstall huggingface_hub datasets

In [28]:
# pip install huggingface_hub datasets

In [ ]:
# Login with Hugging face credentials (need to configure with access token)
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from datasets import load_dataset

# Load the dataset

# Stream the dataset instead of downloading the whole thing
ds = load_dataset("imagenet-1k", streaming=True)

# Code below is the intial code to extract the images and print the labels as is, no perturbations.

In [ ]:
# train_ds = ds['train']
# train_ds_iter = iter(train_ds)

# # Function to display a few images and their labels
# def preview_images(ds_iter, num_samples=5):
#     for _ in range(num_samples):
#         # Get the next sample
#         sample = next(ds_iter)
        
#         # Extract the image and label
#         img = sample['image']  # This is already a PIL image
#         label = sample['label']
        
#         # Display the image
#         img.show()  # This will open the image in the default image viewer
        
#         # Print the label
#         print(f"Label: {label}")

# # Preview the first 5 images and their labels
# preview_images(train_ds_iter, num_samples=5)

# Code below hosts three functions, then prints the resulting images plus the perturbations for 5 images

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


In [ ]:
pip install transformers

In [ ]:
pip install "numpy<2"

In [ ]:
pip install lime

In [ ]:
pip install captum

In [ ]:
pip install opencv-python

In [ ]:
import random
import torch
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from datasets import load_dataset

# Load the pretrained ResNet model and feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")

# Code to Print out the original, un-edited images

In [31]:
import numpy as np
import torch
import cv2
from PIL import Image
from lime import lime_image
import matplotlib.pyplot as plt
from torchvision import transforms
from captum.attr import Saliency

# Set the model to evaluation mode
model.eval()

# Function to dynamically save images with specified names
def save_image(image, name, count):
    plt.imshow(image)
    plt.axis('off')
    plt.savefig(f'image_{name}_{count}.png', bbox_inches='tight', pad_inches=0)
    plt.close()

# Function to generate 10 images per sample
def generate_images(image, count):
    # Save original unperturbed image
    save_image(image, 'original', count)
    
    # Noise Injection with different degrees of perturbation
    stddev_values = [5, 25, 50]  # Small, Moderate, Significant
    for i, stddev in enumerate(stddev_values, 1):
        noisy_image = noise_injection(image, stddev=stddev)
        save_image(noisy_image, f'noise_{stddev}', count)
    
    # Occlusion with different percentages
    occlusion_values = [0.1, 0.3, 0.5]  # Small, Moderate, Significant occlusion
    for i, percentage in enumerate(occlusion_values, 1):
        occluded_image = occlusion(image, percentage=percentage)
        save_image(occluded_image, f'occlusion_{percentage}', count)
    
    # Resolution Reduction with different block sizes
    block_sizes = [5, 10, 20]  # Small, Moderate, Significant reduction
    for i, block_size in enumerate(block_sizes, 1):
        reduced_image = resolution_reduction(image, block_size=block_size)
        save_image(reduced_image, f'resolution_{block_size}', count)

# Process 6 images from the dataset
for count in range(6):
    sample = next(train_ds_iter)
    img = sample['image']  # PIL image
    generate_images(img, count)


# Code to Print out the Explanation Methods Per Image, Per Perturbation Method and its Varying Degrees of Severity

In [ ]:
import numpy as np
import torch
import cv2
from PIL import Image
from lime import lime_image
import matplotlib.pyplot as plt
from torchvision import transforms
from captum.attr import Saliency

# Perturbation functions
def noise_injection(image, distribution='gaussian', mean=0, stddev=25):
    img_array = np.array(image)
    noise = np.random.normal(mean, stddev, img_array.shape) if distribution == 'gaussian' else np.random.uniform(-stddev, stddev, img_array.shape)
    noisy_image = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

def occlusion(image, percentage=0.2):
    img_array = np.array(image)
    height, width, _ = img_array.shape
    occ_height = int(height * percentage)
    occ_width = int(width * percentage)
    top_left_x = np.random.randint(0, width - occ_width)
    top_left_y = np.random.randint(0, height - occ_height)
    img_array[top_left_y:top_left_y + occ_height, top_left_x:top_left_x + occ_width] = 0
    return Image.fromarray(img_array)

def resolution_reduction(image, block_size=2):
    img_array = np.array(image)
    height, width, _ = img_array.shape
    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            block = img_array[i:min(i+block_size, height), j:min(j+block_size, width)]
            avg_color = block.mean(axis=(0, 1)).astype(int)
            img_array[i:min(i+block_size, height), j:min(j+block_size, width)] = avg_color
    return Image.fromarray(img_array)

# Code to Compute and Run Smoothgrad

In [ ]:
# SmoothGrad
def compute_smoothgrad(inputs, model, target_class_idx, n_samples=50, stdev_spread=0.15):
    image_tensor = inputs['pixel_values'].clone().detach().requires_grad_(True)
    saliency = Saliency(lambda noisy_image: model(noisy_image).logits)
    stdev = stdev_spread * (image_tensor.max() - image_tensor.min()).item()
    smooth_grad = torch.zeros_like(image_tensor)
    
    for _ in range(n_samples):
        noise = torch.randn_like(image_tensor) * stdev
        noisy_image = image_tensor + noise
        noisy_saliency = saliency.attribute(noisy_image, target=target_class_idx)
        smooth_grad += noisy_saliency
    
    smooth_grad /= n_samples
    smooth_grad = smooth_grad.squeeze().detach().numpy()
    return smooth_grad

def visualize_smoothgrad(smooth_grad_tensor, image, filename):
    smooth_grad = smooth_grad_tensor.squeeze()
    if smooth_grad.ndim == 3 and smooth_grad.shape[0] == 3:
        smooth_grad = np.mean(smooth_grad, axis=0)
    smooth_grad = (smooth_grad - smooth_grad.min()) / (smooth_grad.max() - smooth_grad.min())
    smooth_grad = np.uint8(255 * smooth_grad)
    heatmap = cv2.applyColorMap(smooth_grad, cv2.COLORMAP_JET)
    original_image = np.array(image)
    if original_image.ndim == 2:
        original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2RGB)
    heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))
    superimposed_image = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_image)
    plt.axis('off')
    plt.savefig(filename)
    plt.close()

# Code to Run LIME

In [ ]:
# LIME
def create_lime_explainer(image, model, feature_extractor):
    explainer = lime_image.LimeImageExplainer()

    def predict_fn(images):
        inputs = feature_extractor(images=[Image.fromarray(img) for img in images], return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return torch.softmax(outputs.logits, dim=1).cpu().numpy()

    # Explain the instance (image) using LIME
    explanation = explainer.explain_instance(
        np.array(image), predict_fn, top_labels=1, hide_color=0, num_samples=50)
    return explanation

def visualize_lime_explanation(image, explanation, filename):
    # Get the image with the mask and apply the explanation mask (positive_only=False will show all regions)
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False
    )

    # Normalize the mask to [0, 1] for applying a colormap
    mask = (mask - mask.min()) / (mask.max() - mask.min())
    
    # Convert the mask to 8-bit values between 0 and 255
    mask = np.uint8(255 * mask)

    # Apply a color map (e.g., COLORMAP_JET or COLORMAP_SUMMER)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

    # Convert the original image to a NumPy array
    original_image = np.array(image)
    if original_image.ndim == 2:  # If grayscale, convert to RGB
        original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2RGB)

    # Resize the heatmap to match the original image size
    heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))

    # Superimpose the heatmap on the original image
    superimposed_image = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0)

    # Save the final result
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_image)
    plt.axis('off')
    plt.savefig(filename)
    plt.close()

# Code to Run the Above Code and Print out Images

In [ ]:
# Save function
def save_image(image, prefix, img_index, perturbation_type, level):
    filename = f"{prefix}_{img_index}_{perturbation_type}_level_{level}.png"
    plt.imshow(image)
    plt.axis('off')
    plt.savefig(filename)
    plt.close()

# Function to generate explanations for the original and perturbed images
def generate_images_and_explanations(image, model, feature_extractor, img_index):
    perturbations = {
        "noise": [noise_injection(image, stddev=stddev) for stddev in [10, 25, 50]],
        "occlusion": [occlusion(image, percentage=percentage) for percentage in [0.1, 0.3, 0.5]],
        "resolution": [resolution_reduction(image, block_size=block_size) for block_size in [5, 10, 20]],
    }

    # Original Image
    save_image(image, 'original', img_index, 'none', 'original')

    # SmoothGrad and LIME for original
    inputs = feature_extractor(images=image, return_tensors="pt")
    smooth_grad_explanation = compute_smoothgrad(inputs, model, target_class_idx=0)
    visualize_smoothgrad(smooth_grad_explanation, image, f"smoothgrad_original_{img_index}.png")
    
    explanation = create_lime_explainer(image, model, feature_extractor)
    visualize_lime_explanation(image, explanation, f"lime_original_{img_index}.png")

    # Perturbed Images
    for perturbation_type, images in perturbations.items():
        for level, perturbed_image in enumerate(images, 1):
            save_image(perturbed_image, 'perturbed', img_index, perturbation_type, level)

            # SmoothGrad for perturbed
            inputs_perturbed = feature_extractor(images=perturbed_image, return_tensors="pt")
            smooth_grad_explanation = compute_smoothgrad(inputs_perturbed, model, target_class_idx=0)
            visualize_smoothgrad(smooth_grad_explanation, perturbed_image, f"smoothgrad_{perturbation_type}_{img_index}_level_{level}.png")

            # LIME for perturbed
            explanation = create_lime_explainer(perturbed_image, model, feature_extractor)
            visualize_lime_explanation(perturbed_image, explanation, f"lime_{perturbation_type}_{img_index}_level_{level}.png")

# Stream the dataset and test explanations
from datasets import load_dataset

# Stream the dataset
ds = load_dataset("imagenet-1k", streaming=True)
train_ds = ds['train']

# Initialize iterator for the 'train' split
train_ds_iter = iter(train_ds)

# Test LIME and SmoothGrad explanations on 6 images
for img_index in range(6):
    sample = next(train_ds_iter)
    img = sample['image']  # PIL image
    generate_images_and_explanations(img, model, feature_extractor, img_index)